# Import dependencies

In [2]:
import os
import sys
import pandas as pd
import re
import pickle
import ast

# Obtener la ruta del directorio padre
parent_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Añadir la ruta del directorio padre al path de Python
sys.path.append(parent_path)
sys.path.append(os.path.join(parent_path, "MongoDB"))
from connect import DatabaseTes
from actions import DataActions
from dataStructure import *

# Read data

## Read source excel

In [3]:
PATH_EXCEL = os.path.join(parent_path,'rawData','TES3Z.xlsx')
df_excel = pd.read_excel(PATH_EXCEL,index_col=0)

In [4]:
data_path = os.path.join(parent_path,'OutputData')
files = os.listdir(data_path)
file = [f for f in files if 'df_raw' in f][0]
file_list = [f for f in files if 'list' in f][0]
file_dict = [f for f in files if 'dict' in f][0]
df_file = os.path.join(data_path,file)
list_path = os.path.join(data_path,file_list)
dict_path = os.path.join(data_path,file_dict)

with open(df_file, 'rb') as handle:
    df = pickle.load(handle)

with open(list_path, 'rb') as handle:
    list_data = pickle.load(handle)

with open(dict_path, 'rb') as handle:
    dict_data = pickle.load(handle)


# Preparing Data

## New document

In [5]:
new_tes = []
for n, row in df.iterrows():
    var = Tes(
        tesName = row['tesName'],
        tags = list(row['tags']),
        fabrication = Fabrication(
            wafer = Wafer(
                waferName = row['tesName'].split('_')[0],
                material = 'Si',
                orientation = '100',
                divisions = ['Q1','Q2','Q3','Q4'],
                chipsOnWafer= [f'Chip_{i}' for i in range(23,77)],
                image = os.path.join(parent_path,'Images','wafer_3Z.jpg')
                ),
            absorber = None if row['absorber'] == 'no' else [Layer(
                    material = 'Au',
                    process = 'electroplating',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = 2, units = 'um'),
                    width = PairValueUnit(quantity = 240, units = 'um'),
                    length = PairValueUnit(quantity = 240, units = 'um')
                ),Layer(
                    material = 'Bi',
                    process = 'electroplating',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = 3.5, units = 'um'),
                    width = PairValueUnit(quantity = 240, units = 'um'),
                    length = PairValueUnit(quantity = 240, units = 'um')
                )],
            membrane = None if row['membrane'] == 'no' else Layer(
                    material = 'Si3N4',
                    process = 'Sputtering',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = 0.5, units = 'um'),
                    width = PairValueUnit(quantity = 250, units = 'um'),
                    length = PairValueUnit(quantity = 250, units = 'um')
                ),
            pads = [Pads(
                material = 'Mo',
                orientation = 'amorphous',
                process = 'sputtering-dc',
                thickness = PairValueUnit(quantity = 15, units = 'um') 
            ),Pads(
                material = 'Nb',
                orientation = 'amorphous',
                process = 'sputtering-dc',
                thickness = PairValueUnit(quantity = 0.15, units = 'um') 
            )],
            stack = [Layer(
                    material = 'Mo',
                    process = 'sputtering-dc',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = row['stackMoT'], units = 'nm'),
                    width = PairValueUnit(quantity = row['stackw'], units = 'um'),
                    length = PairValueUnit(quantity = row['stackl'], units = 'um')
                ),Layer(
                    material = 'Au',
                    process = 'sputtering-dc',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = row['stackAu1T'], units = 'nm'),
                    width = PairValueUnit(quantity = row['stackw'], units = 'um'),
                    length = PairValueUnit(quantity = row['stackl'], units = 'um')
                ),Layer(
                    material = 'Au',
                    process = 'e-beam_evaporation',
                    orintation = 'amorphous',
                    thickness = PairValueUnit(quantity = row['stackAu2T'], units = 'nm'),
                    width = PairValueUnit(quantity = row['stackw'], units = 'um'),
                    length = PairValueUnit(quantity = row['stackl'], units = 'um')
                )]
        )
    ).dict()
    new_tes.append(var)

In [5]:
faltandatos = []
for n, row in df.iterrows():
    for j, caracteristicas in enumerate(row['caracteristicas']):
        for loop in caracteristicas:
            try:
                if row['tags'][1] == 'Funcional':
                    a = dict_data[row['tesName']]
            except Exception as e:
                #print(f'Se produce error {e} en la fila {n}')
                if row['tesName'] not in faltandatos:
                    faltandatos.append(row['tesName'])


In [6]:
faltandatos

['3Z10_74_41']

In [7]:
faltan = []
for i in dict_data.keys():
    if len(df[df['tesName']==i]) == 0 and i not in faltan:
        faltan.append(i)
        


In [8]:
faltan

[]

## New Measurements

In [19]:
df[df['tesName']=='3Z2_53_13'].reset_index(drop = True).iloc[0]['caracteristicas']

[[['3 uA', 'normal', 'redondeo', 'reversible'],
  ['10 uA', 'normal', 'redondeo', 'reversible']]]

In [26]:
tesName = '3Z2_53_13'
list(dict_data[tesName]['dataframe'][0].columns)

['Tem1', 'R(3 uA up)']

In [12]:
datos_ya_introducidos = []
datos_no_introducidos = []
tesName = '3Z2_53_13'
var = dict_data[tesName]
series_foo = df[df['tesName']==tesName].reset_index(drop = True).iloc[0]
for nombre, archivo, corriente, df_valores in zip(var['tesName'], var['dataframe_fileName'], var['current'], var['dataframe']):
    foo = TCmeas(
        fromFile = archivo,
        cryostat = series_foo['cryo'],
        holder = series_foo['holder'],
        primaryType = 
        
    )

{'tesName': ['3Z2_53_13',
  '3Z2_53_13',
  '3Z2_53_13',
  '3Z2_53_13',
  '3Z2_53_13',
  '3Z2_53_13'],
 'dataframe_fileName': ['TES 3Z2_53_13.xls',
  'TES 3Z2_53_13.xls',
  'TES 3Z2_53_13.xls',
  'TES 3Z2_53_13.xls',
  'TES 3Z2_53_13.xls',
  'TES 3Z2_53_13.xls'],
 'current': [3.0, 3.0, 10.0, 10.0, 30.0, 30.0],
 'dataframe': [      Tem1  R(3 uA up)
  0   0.1702     0.00189
  1   0.1703     0.00112
  2   0.1718     0.00062
  3   0.1727    -0.00027
  4   0.1736     0.00094
  ..     ...         ...
  71  0.2429     0.01570
  72  0.2438     0.01687
  73  0.2452     0.01698
  74  0.2467     0.01504
  75  0.2477     0.01622
  
  [76 rows x 2 columns],
         Tem2  R(3 uA down)
  0    0.2518       0.01596
  1    0.2514       0.01737
  2    0.2510       0.01638
  3    0.2508       0.01640
  4    0.2510       0.01604
  ..      ...           ...
  173  0.1781       0.00124
  174  0.1776       0.00141
  175  0.1770       0.00105
  176  0.1765       0.00075
  177  0.1761       0.00200
  
  [178 ro

# Pushing data

In [6]:
objeto = DatabaseTes(True)
objeto.connect()
objeto.collection('tes')
db = objeto.collec

## Insert new documents

In [7]:
dbt_accion = DataActions(db)
db.delete_all_and_insert_many(new_tes)